# Data Exploration: All geo-reconciled data

Internal Data:
- Water Testing lab locations (KENAS Accredited Laboratories)
  - 5 Selected labs
- Selected counties (by named list)

Public Data:
- Kenya ADM Level 0 & 1 (Country outline and County-level) - GADM and COD-AB, choose one
- HOTOSM Populated Places
- UNOCHA COD-PP Populated Places
- GRID3 Settlement Extents (v1.1)

- mWater Waterpoints
- WPDX Waterpoints

---

In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from geopy.distance import distance, lonlat

In [ ]:
from config import primary_counties, secondary_counties, selected_labs, codpp_cols, codpp_county_map

In [ ]:
# A short-list of counties to process for testing and / or baseline investigation
#sel_counties = ["Kilifi", "Nyeri"]
sel_counties = ["Nandi"]

# Otherwise, simply select all primary + secondary
# sel_counties = primary_counties + secondary_counties

sel_counties

## Administrative Boundaries / Base

In [ ]:
adm0_gadm = "./data/gadm41_KEN_0.json"
adm1_gadm = "./data/gadm41_KEN_1.json"

In [ ]:
adm0_df = gpd.read_file(adm0_gadm).to_crs(crs="EPSG:21037")
adm1_df = gpd.read_file(adm1_gadm).to_crs(crs="EPSG:21037")

In [ ]:
# Add county classication (primary, secondary, or nothing) column to select counties
if "COUNTY_CLF" in adm1_df.columns:
    adm1_df["COUNTY_CLF"] = "none"
else:
    adm1_df.insert(11, "COUNTY_CLF", "none")
adm1_df.loc[adm1_df["NAME_1"].isin(primary_counties + [n.replace(" ", "") for n in primary_counties]), "COUNTY_CLF"] = "primary"
adm1_df.loc[adm1_df["NAME_1"].isin(secondary_counties + [n.replace(" ", "") for n in secondary_counties]), "COUNTY_CLF"] = "secondary"

In [ ]:
# Carve out version of ADM1 boundaries for only select counties
sel_adm1_df = adm1_df[adm1_df["NAME_1"].isin(sel_counties + [n.replace(" ", "") for n in sel_counties])].copy()
assert sel_adm1_df.shape[0] == len(sel_counties)
sel_adm1_df.shape

In [ ]:
adm1_df.head(1)

In [ ]:
adm1_df["COUNTY_CLF"].value_counts()

## Water Testing Labs Data (Internal)

In [ ]:
labs_file = "./data/KENAS Accredited Laboratories.xlsx"

In [ ]:
labs_df = pd.read_excel(labs_file, sheet_name="KENAS Water Quality Testing Lab")
labs_df.shape

In [ ]:
labs_df.head(2)

In [ ]:
sel_labs_df = labs_df[labs_df["Laboratory Name"].isin(selected_labs) & labs_df["Location (County)"].isin(primary_counties)].copy()

sel_labs_df = gpd.GeoDataFrame(
    sel_labs_df, geometry=gpd.points_from_xy(
        sel_labs_df["Latitude (Y)"], sel_labs_df["Longitude (X)"], crs="EPSG:21037"))
sel_labs_df.shape

In [ ]:
sel_labs_df

## Populated Places - UNOCHA

In [ ]:
codpp_file = "./data/KEN_Populated places_2002_DEPHA"

In [ ]:
codpp_df = gpd.read_file(codpp_file).to_crs(crs="EPSG:21037")
codpp_df = codpp_df[codpp_cols].copy()
codpp_df["DISTRICT"] = codpp_df["DISTRICT"].replace(codpp_county_map)
codpp_df.shape

In [ ]:
sel_codpp_df = codpp_df[codpp_df["DISTRICT"].isin(sel_counties)].copy()
sel_codpp_df.shape

## Populated Places - HOTOSM

In [ ]:
hotosm_file = "./data/hotosm_ken_populated_places_points_shp.zip"

In [ ]:
hotosm_df = gpd.read_file(hotosm_file).to_crs(crs="EPSG:21037")
hotosm_df = hotosm_df[~hotosm_df["place"].isin(["isolated_dwelling"])]
hotosm_df.shape

In [ ]:
hotosm_df.head(2)

In [ ]:
hotosm_df["place"].value_counts()

In [ ]:
# Filter down to only those in selected AMD1 counties
sel_hotosm_df = hotosm_df[hotosm_df.apply(lambda r: r["geometry"].intersects(sel_adm1_df.geometry).sum() > 0, axis=1)].copy()
sel_hotosm_df.shape

## GRID3 Settlement Extent (SE) Data

***NB: with population***

In [ ]:
se_file = "./data/GRID3_Kenya_Settlement_Extents_Version_1.1/GRID3_Kenya_Settlement_Extents_Version_1.1.gdb/"

In [ ]:
se_df = gpd.read_file(se_file).to_crs(crs="EPSG:21037")
se_df.shape

In [ ]:
# Sub-select to counties (and include all "crosses boundary" items, for accuracy)
sel_se_df = se_df[se_df["adm1_name"].isin(sel_counties + ["crosses boundary"])].copy()
print(f"Filter to counties {sel_counties} and CBs", sel_se_df.shape)

# Drop areas with UN-adjusted population < 500 and > 100,000
pop_low, pop_high = 500, 50000
sel_se_df = sel_se_df[sel_se_df["pop_un_adj"].between(pop_low, pop_high)].copy()
print(f"Filter to {pop_low} < UN Adj Pop < {pop_high}", sel_se_df.shape)

# Further drop any cross-boundary shapes) that do NOT intercept with selected counties
sel_se_df = sel_se_df[sel_se_df.apply(lambda r: r["geometry"].intersects(sel_adm1_df.geometry).sum() > 0, axis=1)].copy()
print("Filter out non-intersecting cross-boundary SEs", sel_se_df.shape)

In [ ]:
sel_se_df.sample()

In [ ]:
#del(se_df)

## PPs within SEs (and within X distance
> NB: with CRS set to EPSG value, distance is in METERS

In [ ]:
# COD-PPs in SEs
sel_codpp_df.apply(lambda r: r["geometry"].within(sel_se_df.geometry).any(), axis=1).sum()

In [ ]:
# HOTOSM-PPs in SEs
sel_hotosm_df.apply(lambda r: r["geometry"].within(sel_se_df.geometry).any(), axis=1).sum()

In [ ]:
# COD-PPs within 1k of SE
dist_threshold = 1000
sel_codpp_df.apply(lambda r: (r["geometry"].distance(sel_se_df.geometry) < dist_threshold).any() , axis=1).sum()

In [ ]:
# HOTOSM-PPs within 1k of SE
sel_hotosm_df.apply(lambda r: (r["geometry"].distance(sel_se_df.geometry) < dist_threshold).any() , axis=1).sum()

In [ ]:
# SEs with PP
sel_se_df.insert(
    sel_se_df.shape[1] - 1,
    "COD_PPs",
    sel_se_df.apply(lambda r: r["geometry"].intersects(sel_codpp_df.geometry).sum(), axis=1))

sel_se_df.insert(
    sel_se_df.shape[1] - 1,
    "HOTOSM_PPs",
    sel_se_df.apply(lambda r: r["geometry"].intersects(sel_hotosm_df.geometry).sum(), axis=1))

sel_se_df.insert(sel_se_df.shape[1] - 1, "HAS_PP", sel_se_df["HOTOSM_PPs"] + sel_se_df["COD_PPs"] > 0)

In [ ]:
# SEs with PP within `dist threshold`
sel_se_df.insert(
    sel_se_df.shape[1] - 1,
    f"COD_PP_WITHIN_{dist_threshold}m",
    sel_se_df.apply(lambda r: (r["geometry"].distance(sel_codpp_df.geometry) < dist_threshold).sum(), axis=1))

sel_se_df.insert(
    sel_se_df.shape[1] - 1,
    f"HOTOSM_PP_WITHIN_{dist_threshold}m",
    sel_se_df.apply(lambda r: (r["geometry"].distance(sel_hotosm_df.geometry) < dist_threshold).sum(), axis=1))

sel_se_df.insert(
    sel_se_df.shape[1] - 1,
    f"PP_WITHIN_{dist_threshold}m",
    sel_se_df[f"COD_PP_WITHIN_{dist_threshold}m"] + sel_se_df[f"HOTOSM_PP_WITHIN_{dist_threshold}m"] > 0)

## SE (Community w/populations) stats

In [ ]:
# Num SEs
sel_se_df.shape[0]

In [ ]:
sel_se_df[["population", "pop_un_adj"]].describe()

In [ ]:
px.histogram(sel_se_df, x="pop_un_adj")

In [ ]:
sel_se_df[sel_se_df["HAS_PP"] | sel_se_df["PP_WITHIN_1000m"]]

## Explore Map

In [ ]:
# Map Country and County Outlines
def county_shape_style(x):
    """Style KWDS:
    stroke: bool (default True) Outline
    color: string Stroke color
    weight: int Stroke width in pixels
    opacity: float (default 1.0) Stroke opacity
    fill: bool (default True) Whether to fill
    fillColor: str
    fillOpacity: float (default 0.5)
    """
    if x["properties"]["COUNTY_CLF"] == "primary":
        return dict(color="red", weight=2, opacity=0.75, fill=True, fillOpacity=0.05)
    elif x["properties"]["COUNTY_CLF"] == "secondary":
        return dict(color="orange", weight=2, opacity=0.75, fill=True, fillOpacity=0.05)
    else:
        return dict(stroke=False, fill=False)

disp_cols = ["COUNTRY", "NAME_1", "COUNTY_CLF"]
m = adm0_df.explore(style_kwds=dict(color="navy", opacity=0.5, fill=False), tooltip=False, tiles="CartoDB positron")
m = adm1_df.explore(m=m, style_kwds=dict(style_function=county_shape_style), tooltip=False, popup=disp_cols, highlight=False)

In [ ]:
# Add GRID3-SEs to map
disp_cols = ['Shape_Length', 'Shape_Area', 'type', 'population', 'pop_un_adj', 'adm1_name', 'settl_pcode']
m = sel_se_df.explore(
    # "pop_un_adj",
    "mgrs_code",
    cmap="tab20",
    # color="paleturquoise",
    # cmap="Blues",
    # vmin=0,
    # vmax=200000,
    style_kwds=dict(color="black", weight=0.5, fillOpacity=0.3),
    tooltip=disp_cols,
    popup=False,
    legend=False,
    m=m,)

In [ ]:
# Add Water Testing Labs to map
disp_cols = ['Laboratory Name', 'Location (County)', 'Laboratory Type', 'Accreditation Expiry Date', 'Contact']
m = sel_labs_df.explore(m=m, color="limegreen", marker_kwds=dict(radius=6), tooltip=disp_cols, popup=disp_cols)

In [ ]:
# Add COD-PPs to map
m = sel_codpp_df.explore(m=m, color="dodgerblue")

In [ ]:
# Add HOTOSM-PP to map
m = sel_hotosm_df.explore(m=m, color="mediumorchid")